##**Atividade Avaliativa - Processamento de Linguagem Natural**
### **Equipe:** Ana Clara, Bruno, Igor e Pedro Cruz


# **Descrição e motivação do problema:**

A disseminação de **fake news** tem acontecido com cada vez mais frequência e possui impacto significativo na sociedade, contribuindo para a desinformação, polarização social e decisões equivocadas. A motivação para desenvolver um algoritmo de identificação de notícias falsas está em promover uma sociedade mais informada, fornecendo as pessoas uma ferramenta automatizada para avaliar a veracidade de informações rapidamente. Isso pode ajudar a mitigar os efeitos negativos da desinformação, capacitando as pessoas a tomarem decisões mais conscientes e baseadas em fatos confiáveis.

# **Descrição da base de dados (e do processo de obtenção dos dados, se for o caso):**

A base de dados que decidimos utilizar é a [Fake.Br Corpus](https://github.com/roneysco/Fake.br-Corpus). Ela fornece pares de notícias verdadeiras e falsas em português.

Mais especificamente, iremos utilizar os dados contidos na pasta "size_normalized_texts", que contém versões truncadas dos textos, onde em cada par verdadeiro-falso o texto mais longo é truncado (em número de palavras) para o tamanho do texto mais curto. Esta versão do corpus pode ser útil para evitar inclinações incorretas em experimentos de aprendizado de máquina.

# **Objetivo de negócio ou científico associado ao problema:**


O objetivo científico deste projeto é desenvolver e validar um algoritmo capaz de identificar fake news de algum assunto específico com precisão. Já o objetivo de negócio é fornecer uma solução automatizada, como um chatbot, que permita organizações, jornalistas e usuários individuais verificarem rapidamente a veracidade de informações, reduzindo os impactos da desinformação em escala e promovendo a confiança em fontes confiáveis.

# **Etapa de limpeza e pré-processamento:**

Unzipando os arquivos baixados

In [2]:
 import zipfile

 caminho_zip = 'Fake.br-Corpus-master.zip'

 with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
  zip_ref.extractall('arquivos')

Obtendos os dados da pasta size_normalized_texts, como comentado anteriormente, e colocando tudo em um dataframe, inserindo o label 0 ou 1 para distinguirmos entre verdadeiro ou falso.

In [3]:
import os
import pandas as pd

fake_dir = 'arquivos/Fake.br-Corpus-master/size_normalized_texts/fake'
true_dir = 'arquivos/Fake.br-Corpus-master/size_normalized_texts/true'

def load_texts(directory, label):
    data = []
    for file in os.listdir(directory):
        with open(os.path.join(directory, file), 'r', encoding='utf-8') as f:
            data.append({'text': f.read(), 'label': label})
    return data

fake_data = load_texts(fake_dir, 0)  # 0 para "falso"
true_data = load_texts(true_dir, 1)  # 1 para "verdadeiro"

df = pd.DataFrame(fake_data + true_data)

print(df.head())

                                                text  label
0  Ele não tem medo de altura! "Chuck Norris" da ...      0
1  Jandira "intitulou" Aécio de traficante. Aécio...      0
2  Senador Bauer: "Quem quiser ver uma grande obr...      0
3  Renan afronta STF e se recusa a deixar a presi...      0
4  Ministro da Fazenda promete abrir a "caixa pre...      0


Exploração

In [4]:
df.shape

(7200, 2)

Verificando se temos algum nulo

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7200 entries, 0 to 7199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7200 non-null   object
 1   label   7200 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 112.6+ KB


Pré-processamento

In [17]:
 import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

stop_words = set(stopwords.words('portuguese'))

def preprocess_text(text):
    words = word_tokenize(text.lower(), language='portuguese')
    words = [word for word in words if word.isalpha() and word not in stop_words]
    return ' '.join(words)

df['processed_text'] = df['text'].apply(preprocess_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
df.head()

,text,label,processed_text
0,"Ele não tem medo de altura! ""Chuck Norris"" da ...",0,medo altura chuck norris bahia instala tela pr...
1,"Jandira ""intitulou"" Aécio de traficante. Aécio...",0,jandira intitulou aécio traficante aécio proto...
2,"Senador Bauer: ""Quem quiser ver uma grande obr...",0,senador bauer quiser ver grande obra pt viajar...
3,Renan afronta STF e se recusa a deixar a presi...,0,renan afronta stf recusa deixar presidência se...
4,"Ministro da Fazenda promete abrir a ""caixa pre...",0,ministro fazenda promete abrir caixa preta pt ...


# **Extração de características:**

# **Modelos de Machine Learning:**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['processed_text']).toarray()
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# **Protocolo de experimentos e validação**

# **Discussão dos resultados e trabalhos futuros**